<CENTER><img src="../../images/ATLASOD.gif" style="width:50%"></CENTER>

In [93]:

import uproot # for reading .root files
import time # to measure time to analyse
import math # for mathematical functions such as square root

#import awkward as ak # for handling complex and nested data structures efficiently

import pandas as pd

import numpy as np # # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import MaxNLocator,AutoMinorLocator # for minor ticks
from lmfit.models import PolynomialModel, GaussianModel # for the signal and background fits
import vector #to use vectors
import requests # for HTTP access
import aiohttp # HTTP client support

In [94]:
# ATLAS Open Data directory
path = "https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/" # web address
tuple_path = "/project/etp1/gduckeck/atlas/13TeV/GamGam/Data/" #Current path!
tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/" # web address

#tuple_path = \"~/Downloads/GamGamNew/\" # local

In [95]:
samples_list = ['data15_periodD','data15_periodE','data15_periodF','data15_periodG','data15_periodH','data15_periodJ','data16_periodA','data16_periodB','data16_periodC','data16_periodD','data16_periodE','data16_periodF','data16_periodG','data16_periodK','data16_periodL']
samples_list = ['data_A', 'data_B', 'data_C','data_D'] # add if you want more data

In [96]:
print(samples_list[1])

# This is now appended to our file path to retrieve the data15_periodG.root file
data_B_path = path + samples_list[3] + ".GamGam.root"

data_B


In [97]:
data_B_path

'https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/data_D.GamGam.root'

In [98]:
# Accessing the file from the online database (":analysis" opens the tree in a desired manner)
for sample in samples_list:
    # Construct the file path
    file_path = f"{path}{sample}.GamGam.root" #e.g. tree = .../GamGam/Data/data15_periodD.root + :analysis
    
    try:
        # Open the tree named ":analysis" in each file
        with uproot.open(file_path + ": mini") as t:
            tree = t
            
            # Print the number of entries in the tree
            print(f"Processing {sample}:")
            print(f"Number of entries in the tree: {tree.num_entries}")
            
            # Print the information stored in the tree
         
            
            # Perform any additional operations on the tree here
            # For example, applying cuts, extracting data, etc.
            # ...
            
    except Exception as e:
        print(f"Failed to process {sample}. Error: {e}")


# The number of entries in the tree can be viewed
print("The number of entries in the tree are:", tree.num_entries)

# All the information stored in the tree can be viewed using the .keys() method.
print("The information stored in the tree is:", tree.keys())

Failed to process data_A. Error: https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/data_A.GamGam.root
Failed to process data_B. Error: https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/data_B.GamGam.root
Failed to process data_C. Error: https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/data_C.GamGam.root
Failed to process data_D. Error: https://atlas-opendata.web.cern.ch/atlas-opendata/13TeV/GamGam/Data/data_D.GamGam.root
The number of entries in the tree are: 3602176
The information stored in the tree is: ['runNumber', 'eventNumber', 'channelNumber', 'mcWeight', 'scaleFactor_PILEUP', 'scaleFactor_ELE', 'scaleFactor_MUON', 'scaleFactor_PHOTON', 'scaleFactor_TAU', 'scaleFactor_BTAG', 'scaleFactor_LepTRIGGER', 'scaleFactor_PhotonTRIGGER', 'trigE', 'trigM', 'trigP', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_pt', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_charge', 'lep_type', 'lep_isTightID', 'lep_ptcone30', 'lep_etcone20'

Perhaps we'd like to see the energies of the photons. 
We can access this from our tree using the key `photon_e`.

In [99]:
variables = ["photon_pt", "photon_eta", "photon_phi", "photon_E",
                                  "photon_isTightID", "photon_etcone20"]
fraction = 1 #lower to deload data

In [100]:
def calc_myy(photon_pt,photon_eta,photon_phi,photon_E):
    # first photon is [0], 2nd photon is [1] etc
    px_0 = photon_pt[0]*math.cos(photon_phi[0]) # x-component of photon[0] momentum
    py_0 = photon_pt[0]*math.sin(photon_phi[0]) # y-component of photon[0] momentum
    pz_0 = photon_pt[0]*math.sinh(photon_eta[0]) # z-component of photon[0] momentum
    px_1 = photon_pt[1]*math.cos(photon_phi[1]) # x-component of photon[1] momentum
    py_1 = photon_pt[1]*math.sin(photon_phi[1]) # y-component of photon[1] momentum
    pz_1 = photon_pt[1]*math.sinh(photon_eta[1]) # z-component of photon[1] momentum
    sumpx = px_0 + px_1 # x-component of diphoton momentum
    sumpy = py_0 + py_1 # y-component of diphoton momentum
    sumpz = pz_0 + pz_1 # z-component of diphoton momentum 
    sump = math.sqrt(sumpx**2 + sumpy**2 + sumpz**2) # magnitude of diphoton momentum 
    sumE = photon_E[0] + photon_E[1] # energy of diphoton system
    m2 = sumE**2 - sump**2
    if m2>0.:
        m = m2**0.5
    else:
        print('calc_myy error neg mass**2', m2)
        m = 0.
    return m 

Based on our analysis, this entry should be removed because the photons do not match all our requirements.
We can turn these checks and calculations into a set of functions.

In [101]:
def cut_photon_reconstruction(photon_isTightID):
    return photon_isTightID[0]==True and photon_isTightID[1]==True

def cut_photon_pt(photon_pt):
    return photon_pt[0]>40000 and photon_pt[1]>30000

def cut_photon_eta_transition(photon_eta):
    return (abs(photon_eta[0])>1.52 or abs(photon_eta[0])<1.37) and (abs(photon_eta[1])>1.52 or abs(photon_eta[1])<1.37)

def cut_n_photon(photon_n):
    return photon_n>=2

def cut_isolation_et(photon_etcone20):
    # Keep only the events where first two photons have isolation pt < 4 GeV
    return (photon_etcone20[0] < 4000) and (photon_etcone20[1] < 4000)




You may verify on your own that these functions give the same outputs as the previous code block.
Now, 
    we shall apply these functions over the entire data tree using a `for` loop.

In [102]:
def get_data_from_files():
    frames = []  # define empty list to hold data
    for val in samples_list:  # loop over each file
        fileString = tuple_path + val + ".GamGam.root"  # file name to open
        temp = read_file(fileString)  # call the function read_file defined earlier
        frames.append(temp)  # append DataFrame returned from read_file to list of DataFrames

    # Concatenate list of pandas DataFrames
    data = pd.concat(frames, ignore_index=True)  
    
    return data  # return concatenated DataFrame


In [129]:
def read_file(path):
    start = time.time()
    print("Processing: " + path)
    data_all = pd.DataFrame()
    tree = uproot.open(path + ":mini")
    numevents = tree.num_entries
    
    total_events = 0
    cutflow = {
        'initial': 0,
        'tight_id': 0,
        'pt_cut': 0,
        'isolation': 0,
        'eta_cut': 0
    }

    for data in tree.iterate(["photon_pt", "photon_eta", "photon_phi", "photon_E",
                              "photon_isTightID", "photon_etcone20"],
                             library="pd",
                             entry_stop=numevents*fraction):
        nIn = len(data.index)
        total_events += nIn
        cutflow['initial'] += nIn

        print("before cut", len(data.index))
        data = data[np.vectorize(cut_photon_reconstruction)(data.photon_isTightID)]
        cutflow['tight_id'] += len(data.index)
        print("istight-cut", len(data.index))

        data = data[np.vectorize(cut_photon_pt)(data.photon_pt)]
        cutflow['pt_cut'] += len(data.index)
        print("pt-cut", len(data.index))

        data = data[np.vectorize(cut_isolation_et)(data.photon_etcone20)]
        cutflow['isolation'] += len(data.index)
        print("iso-cut", len(data.index))

        data = data[np.vectorize(cut_photon_eta_transition)(data.photon_eta)]
        cutflow['eta_cut'] += len(data.index)
        print("eta-cut", len(data.index))

        data['myy'] = np.vectorize(calc_myy)(data.photon_pt, data.photon_eta, data.photon_phi, data.photon_E)

        nOut = len(data.index)
        data_all = pd.concat([data_all, data], ignore_index=True)
        elapsed = time.time() - start
        print(f"\t nIn: {nIn},\t nOut: \t{nOut}\t in {round(elapsed,1)}s")

    print("\nCutflow Summary:")
    print(f"{'Cut':<20}{'Events':<10}{'Percentage':<10}")
    print("-" * 40)
    for cut, count in cutflow.items():
        percentage = (count / total_events) * 100
        print(f"{cut:<20}{count:<10}{percentage:.2f}%")

    return data_all

In [130]:
start = time.time() # time at start of whole processing
data = get_data_from_files() # process all files
elapsed = time.time() - start # time after whole processing
print("Time taken: "+str(round(elapsed,1))+"s") # print total time taken to process every file

Processing: https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/data_A.GamGam.root
before cut 430344
istight-cut 54379
pt-cut 34625
iso-cut 22381
eta-cut 22350
	 nIn: 430344,	 nOut: 	22350	 in 3.9s

Cutflow Summary:
Cut                 Events    Percentage
----------------------------------------
initial             430344    100.00%
tight_id            54379     12.64%
pt_cut              34625     8.05%
isolation           22381     5.20%
eta_cut             22350     5.19%
Processing: https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/data_B.GamGam.root


KeyboardInterrupt: 

We are now ready to fit our data to effectively detect the Higgs boson! We will use a combination of a 4th order polynomial and a Gaussian function. The polynomial function represents the background, while the Gaussian function represents our signal. The Gaussian model is used to fit the signal due to the nature of the detector's resolution. The fourth-order polynomial is chosen for the background because it offers enough flexibility to capture the overall shape without overfitting, thereby reducing the influence of spurious data—random, irrelevant fluctuations or noise that do not correspond to the true signal or background.

In [ ]:
len(data)

In [ ]:
data['myy']

In [ ]:
import awkward as ak



data_x,_ = np.histogram(ak.to_numpy(data['myy']), 
                            bins=bin_edges ) # histogram the data
data_x_errors = np.sqrt( data_x ) # statistical error on the data

# data fit
polynomial_mod = PolynomialModel( 4 ) # 4th order polynomial
gaussian_mod = GaussianModel() # Gaussian

# set initial guesses for the parameters of the polynomial model
# c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
pars = polynomial_mod.guess(data_x, # data to use to guess parameter values
                            x=bin_centres, c0=data_x.max(), c1=0,
                            c2=0, c3=0, c4=0 )

# set initial guesses for the parameters of the Gaussian model
pars += gaussian_mod.guess(data_x, # data to use to guess parameter values
                        x=bin_centres, amplitude=100, 
                        center=125, sigma=2 )

model = polynomial_mod + gaussian_mod # combined model

# fit the model to the data
out = model.fit(data_x, # data to be fit
                pars, # guesses for the parameters
                x=bin_centres, weights=1/data_x_errors ) #ASK

# background part of fit
params_dict = out.params.valuesdict() # get the parameters from the fit to data
c0 = params_dict['c0'] # c0 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
c1 = params_dict['c1'] # c1 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
c2 = params_dict['c2'] # c2 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
c3 = params_dict['c3'] # c3 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4
c4 = params_dict['c4'] # c4 of c0 + c1*x + c2*x^2 + c3*x^3 + c4*x^4

# get the background only part of the fit to data
background = c0 + c1*bin_centres + c2*bin_centres**2 + c3*bin_centres**3 + c4*bin_centres**4

# data fit - background fit = signal fit
signal_x = data_x - background 

# *************
# Main plot 
# *************
plt.axes([0.1,0.3,0.85,0.65]) # left, bottom, width, height 
main_axes = plt.gca() # get current axes

# plot the data points
main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors, 
                fmt='ko', # 'k' means black and 'o' means circles
                label='Data' ) 

# plot the signal + background fit
main_axes.plot(bin_centres, # x
            out.best_fit, # y
            '-r', # single red line
            label='Sig+Bkg Fit ($m_H=125$ GeV)' )

# plot the background only fit
main_axes.plot(bin_centres, # x
            background, # y
            '--r', # dashed red line
            label='Bkg (4th order polynomial)' )

# set the x-limit of the main axes
main_axes.set_xlim( left=xmin, right=xmax ) 

# separation of x-axis minor ticks
main_axes.xaxis.set_minor_locator( AutoMinorLocator() ) 

# set the axis tick parameters for the main axes
main_axes.tick_params(which='both', # ticks on both x and y axes
                    direction='in', # Put ticks inside and outside the axes
                    top=True, # draw ticks on the top axis
                    labelbottom=False, # don't draw tick labels on bottom axis
                    right=True ) # draw ticks on right axis

# write y-axis label for main 
main_axes.set_ylabel('Events / '+str(step_size)+' GeV', 
                    horizontalalignment='right') 

# set the y-axis limit for the main axes
main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.1 ) 

# set minor ticks on the y-axis of the main axes
main_axes.yaxis.set_minor_locator( AutoMinorLocator() ) 

# avoid displaying y=0 on the main axes
main_axes.yaxis.get_major_ticks()[0].set_visible(False) 

# Add text 'ATLAS Open Data' on plot
plt.text(0.2, # x
        0.92, # y
        'ATLAS Open Data', # text
        transform=main_axes.transAxes, # coordinate system used is that of main_axes
        fontsize=13 ) 

# Add text 'for education' on plot
plt.text(0.2, # x
        0.86, # y
        'for education', # text
        transform=main_axes.transAxes, # coordinate system used is that of main_axes
        style='italic',
        fontsize=8 ) 

lumi = 36.1
lumi_used = str(lumi*fraction) # luminosity to write on the plot
plt.text(0.2, # x
        0.8, # y
        '$\sqrt{s}$=13 TeV,$\int$L dt = '+lumi_used+' fb$^{-1}$', # text
        transform=main_axes.transAxes ) # coordinate system used is that of main_axes 

# Add a label for the analysis carried out
plt.text(0.2, # x
        0.74, # y
        r'$H \rightarrow \gamma\gamma$', # text 
        transform=main_axes.transAxes ) # coordinate system used is that of main_axes

# draw the legend
main_axes.legend(frameon=False, # no box around the legend
                loc='lower left' ) # legend location 


# *************
# Data-Bkg plot 
# *************
plt.axes([0.1,0.1,0.85,0.2]) # left, bottom, width, height
sub_axes = plt.gca() # get the current axes

# set the y axis to be symmetric about Data-Background=0
sub_axes.yaxis.set_major_locator( MaxNLocator(nbins='auto', 
                                            symmetric=True) )

# plot Data-Background
sub_axes.errorbar(x=bin_centres, y=signal_x, yerr=data_x_errors,
                fmt='ko' ) # 'k' means black and 'o' means circles

# draw the fit to data
sub_axes.plot(bin_centres, # x
            out.best_fit-background, # y
            '-r' ) # single red line

# draw the background only fit
sub_axes.plot(bin_centres, # x
            background-background, # y
            '--r' )  # dashed red line

# set the x-axis limits on the sub axes
sub_axes.set_xlim( left=xmin, right=xmax ) 

# separation of x-axis minor ticks
sub_axes.xaxis.set_minor_locator( AutoMinorLocator() ) 

# x-axis label
sub_axes.set_xlabel(r'di-photon invariant mass $\mathrm{m_{\gamma\gamma}}$ [GeV]',
                    x=1, horizontalalignment='right', 
                    fontsize=13 ) 

# set the tick parameters for the sub axes
sub_axes.tick_params(which='both', # ticks on both x and y axes
                    direction='in', # Put ticks inside and outside the axes
                    top=True, # draw ticks on the top axis
                    right=True ) # draw ticks on right axis 

# separation of y-axis minor ticks
sub_axes.yaxis.set_minor_locator( AutoMinorLocator() ) 

# y-axis label on the sub axes
sub_axes.set_ylabel( 'Events-Bkg' ) 


# Generic features for both plots
main_axes.yaxis.set_label_coords( -0.09, 1 ) # x,y coordinates of the y-axis label on the main axes
sub_axes.yaxis.set_label_coords( -0.09, 0.5 ) # x,y coordinates of the y-axis label on the sub axes make it pandas eligible

In [ ]:
params_dict

And there it is—a clear peak in the invariant mass spectrum around 125 GeV, signaling the presence of the Higgs boson! While our main task may be done, there's still more to explore. Here are some additional tasks you can try with this notebook:

* Check how many events are being thrown away by each cut in '[Applying a cut](#applying_cut)'
* Add more cuts from the [Higgs discovery paper](https://www.sciencedirect.com/science/article/pii/S037026931200857X#se0090) in '[Changing a cut](#changing_cut)' and '[Applying a cut](#applying_cut)'
* Find the reduced chi-squared for the fit in '[Plotting](#plotting)'
* Find the mean of the fitted Gaussian in '[Plotting](#plotting)'
* Find the width of the fitted Gaussian in '[Plotting](#plotting)'
* Try different initial guesses for the parameters of the fit in '[Plotting](#plotting)'
* Try different functions for the fit in '[Plotting](#plotting)'
* Your idea!